In [ ]:
import pandas as pd
import numpy as np
import cv2
from matplotlib import pyplot as plt
from sklearn.cluster import estimate_bandwidth, MeanShift
from PIL import Image

def boundary(path_model, path_train, boundary_pts, size=(12,3)):
    # Drawing the bounding box
    img_train = cv2.imread(path_train) 
    img_rgb_train = cv2.cvtColor(img_train,cv2.COLOR_BGR2RGB)
    img_rgb_train = cv2.polylines(img_rgb_train,boundary_pts,True,(0,180,0),8, cv2.LINE_AA)
    
    cof=(int(sum(boundary_pts[0][i][0][0] for i in range(0, 4))/4),int(sum(boundary_pts[0][i][0][1] for i in range(0, 4))/4))
    shape=(min(abs(boundary_pts[0][0][0][0]-boundary_pts[0][3][0][0]),abs(boundary_pts[0][1][0][0]-boundary_pts[0][2][0][0])),min(abs(boundary_pts[0][1][0][1]-boundary_pts[0][0][0][1]),abs(boundary_pts[0][2][0][1]-boundary_pts[0][3][0][1])))
    img_rgb_train = cv2.circle(img_rgb_train, cof, radius=10, color=(255, 0, 0), thickness=-1)
    
    img_model = cv2.imread(path_model) 
    img_rgb_model = cv2.cvtColor(img_model,cv2.COLOR_BGR2RGB)
    
    print("{Position: %s, width: %d, height: %d}" % (str(cof),shape[0],shape[1]))
    
    f, axarr = plt.subplots(1,2,figsize=size)
    axarr[0].imshow(img_rgb_train)
    axarr[1].imshow(img_rgb_model)
    plt.show()
    
    
def show_image(path, size=(20,4)):
    img = cv2.imread(path)   
    img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    plt.figure(figsize=size)
    plt.imshow(img_rgb)
    plt.show()
    

In [ ]:
##LOCAL OPERATOR##

#Gaussian
def gaussian_filter(path, sigma=1.5):
    k_size = int(np.ceil((3*sigma))*2 + 1) 
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img_tr = cv2.GaussianBlur(img, (k_size,k_size) , sigma)
    return img_tr

#Bilateral Filter
def bilateral_filter(path, k_size = 9, sigma1= 75, sigma2=75):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img_bilateral = cv2.bilateralFilter(img,9,75,75)
    return img_bilateral

In [ ]:
def sift_cluster_recognition(path_model, sigma_model, path_train, sigma_train, min_match_count, quantile):
    
    #Gaussian
    img_model_bw = gaussian_filter(path_model, sigma_model)
    img_train_bw = gaussian_filter(path_train, sigma_train)
            
    #Bilateral
    #img_model_bw = bilateral_filter(path_model)
    #img_train_bw = bilateral_filter(path_train)
    
    sift = cv2.xfeatures2d.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img_model_bw, None)
    kp2, des2 = sift.detectAndCompute(img_train_bw, None)

    x = np.array([kp2[0].pt])

    for i in range(len(kp2)):
        x = np.append(x, [kp2[i].pt], axis=0)

    x = x[1:len(x)]

    bandwidth = estimate_bandwidth(x, quantile=quantile, n_samples=None, random_state=0)

    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True, cluster_all=True)
    ms.fit(x)
    labels = ms.labels_
    cluster_centers = ms.cluster_centers_

    labels_unique = np.unique(labels)
    n_clusters_ = len(labels_unique)

    s = [None] * n_clusters_
    for i in range(n_clusters_):
        l = ms.labels_
        d, = np.where(l == i)
        s[i] = list(kp2[xx] for xx in d)

    des2_ = des2

    for i in range(n_clusters_):

        kp2 = s[i]
        l = ms.labels_
        d, = np.where(l == i)
        des2 = des2_[d, ]

        FLANN_INDEX_KDTREE = 0
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks = 50)

        flann = cv2.FlannBasedMatcher(index_params, search_params)

        des1 = np.float32(des1)
        des2 = np.float32(des2)

        matches = flann.knnMatch(des1, des2, 2)

        # store all the good matches as per Lowe's ratio test.
        good = []
        for m,n in matches:
            if m.distance < 0.7*n.distance:
                good.append(m)

        if len(good)>min_match_count:
            src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
            dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 2)

            if M is None:
                print ("No Homography")
            else:
                print ("Found in cluster %d the image %s - %d/%d" % (i, path_model,len(good),min_match_count))
                matchesMask = mask.ravel().tolist()
                h,w = img_model_bw.shape
                pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
                dst = cv2.perspectiveTransform(pts,M)
                boundary_pts = [np.int32(dst)]
                boundary(path_model, path_train, boundary_pts, size =(80,16))
                #Eliminare il commento se si desidera eseguire il training per la parameters optimization
                #createTable(path_train, path_model, boundary_pts, quantile, sigma_model, sigma_train, min_match_count, len(good))
        #else:
            #print ("Not found in cluster %d the image %s - %d/%d" % (i, path_model,len(good),min_match_count))


TASK A
USING SIFT

In [ ]:
imgs_train = ['scenes/e1.png', 'scenes/e2.png', 'scenes/e3.png', 'scenes/e4.png', 'scenes/e5.png']
imgs_model = ['models/0.jpg', 'models/1.jpg', 'models/11.jpg', 'models/19.jpg', 'models/24.jpg', 'models/26.jpg', 'models/25.jpg']

for path_train in imgs_train:
    print("In the scene: ")
    show_image(path_train, (20,4))
    for path_model in imgs_model:
        sift_cluster_recognition(path_model, 2.5, path_train, 1.5, 75, 1)

TASK B USING SIFT AND CLUSTERING

In [ ]:
imgs_train = ['scenes/m1.png', 'scenes/m2.png', 'scenes/m3.png', 'scenes/m4.png', 'scenes/m5.png']
imgs_model = ['models/0.jpg', 'models/1.jpg', 'models/11.jpg', 'models/19.jpg', 'models/24.jpg', 'models/26.jpg', 'models/25.jpg']

for path_train in imgs_train:
    print("In the scene: ")
    show_image(path_train, (20,4))
    for path_model in imgs_model:
        sift_cluster_recognition(path_model, 2.0, path_train, 1.5, 90, 0.15)        

TASK C BONUS
USING SIFT AND CLUSTERING

In [ ]:
imgs_train = ['scenes/h1.jpg','scenes/h2.jpg', 'scenes/h3.jpg', 'scenes/h4.jpg', 'scenes/h5.jpg']

for path_train in imgs_train:
    print("In the scene: ")
    show_image(path_train)
    for i in range(0, 24):
        sift_cluster_recognition("models/"+str(i)+".jpg", 2.5, path_train, 0.4, 14, 0.035)

PARAMETERS OPTIMIZATION 
TRAINING IN TASK C

In [ ]:
height_img_train = 480
width_img_train = 640
table = pd.DataFrame(columns=['Scenes', 'Models', 'Quantile', 'Sigma_Model', 'Sigma_Train', 'MinMatchCount', 'Match', 'TP', 'FP'])
table.to_csv("attempts/attempts_4.csv", mode='a',index=False, header=True)

def createTable(path_train, path_model, boundary_pts, quantile, sigma_model, sigma_train, min_match_count, good):
    
    table = pd.DataFrame(columns=['Scenes', 'Models', 'Quantile', 'Sigma_Model', 'Sigma_Train', 'MinMatchCount', 'Match', 'TP', 'FP'])

    numCSV = path_train[8]
    path_csv = "centerScenes/h"+numCSV+".csv"
    df = pd.read_csv(path_csv, sep = ',', error_bad_lines=False) 
     
    cof=(int(sum(boundary_pts[0][i][0][0] for i in range(0, 4))/4),int(sum(boundary_pts[0][i][0][1] for i in range(0, 4))/4))
    
    
    df_Model = df[(df["Model"] == i)]
    ok = 0
    for row in range(0,int(df_Model.shape[0])):
        
        x_Center = int(df_Model.iloc[row,1]*width_img_train) 
        y_Center = int(df_Model.iloc[row,2]*height_img_train)
        width = int(df_Model.iloc[row,3]*width_img_train)
        height = int(df_Model.iloc[row,4]*height_img_train)
        
        x1 = x_Center - (width/2)
        x2 = x_Center + (width/2)
        y1 = y_Center + (height/2)
        y2 = y_Center - (height/2)
        
        if cof[0] < int(x2): 
            if cof[0] > int(x1):
                if cof[1] < int(y1):
                    if cof[1] > int(y2):
                        ok = 1
                  
    if ok == 1:
        truePositive = 1
        falsePositive = 0
    else:
        truePositive = 0
        falsePositive = 1
        
    newRow = { 'Scenes' : path_train, 'Models' : path_model, 'Quantile': quantile, 'Sigma_Model': sigma_model, 'Sigma_Train': sigma_train, 'MinMatchCount' : min_match_count, 'Match' : good, 'TP' : int(truePositive), 'FP' : int(falsePositive)}
    table = table.append(newRow, ignore_index=True)
    table.to_csv("attempts/attempts_4.csv", mode='a',index=False, header=False)       

for j in range(1,6):
    path_train = "scenes/h"+str(j)+".jpg"
    print("In the scene: "+path_train)
    for quantile in range(10,40,5):
        for sigma_model in range(25,36,5):
            for sigma_train in range(25,41,5):
                for minMatch in range(13,15):
                    for i in range(0, 24):
                        quant = quantile/1000
                        sigma_model_f = sigma_model/10
                        sigma_train_f = sigma_train/100
                        sift_cluster_recognition("models/"+str(i)+".jpg", sigma_model_f, "scenes/h"+str(j)+".jpg", sigma_train_f, minMatch, quant)

In [ ]:
rank = pd.DataFrame(columns=['Quantile', 'Sigma_Model', 'Sigma_Train', 'MinMatchCount', 'Sum TP', 'Sum FP'])

df = pd.read_csv('attempts/attempts_3.csv', sep = ',', error_bad_lines=False) 
quantile = df["Quantile"].unique()
sigma_model = df["Sigma_Model"].unique()
sigma_Train = df["Sigma_Train"].unique()
minMatchCount = df["MinMatchCount"].unique()
for quant in quantile:
    for sigmaM in sigma_model:
        for sigmaT in sigma_Train:
            for minMC in minMatchCount:
                df_Parameters = df[(df['Quantile'] == quant) & (df['Sigma_Model'] == sigmaM) & (df['Sigma_Train'] == sigmaT)  & (df['MinMatchCount'] == minMC)]
                sumTP = df_Parameters['TP'].sum()
                sumFP = df_Parameters['FP'].sum()
                relationship = sumTP - sumFP
                newRow = { 'Quantile' : quant, 'Sigma_Model' : sigmaM, 'Sigma_Train': sigmaT, 'MinMatchCount' : minMC, 'Sum TP' : sumTP, 'Sum FP' : sumFP, 'Relationship' : relationship}
                rank = rank.append(newRow, ignore_index=True)
                rank = rank.sort_values(by='Relationship', ascending=False)

rank.to_csv("rankings/rank_4.csv", mode='w',index=False, header=True)